# Day 0: Imports and Utility Functions

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os
import re
import numpy as np
import random
import string
import itertools
from collections import Counter, defaultdict, namedtuple, deque, OrderedDict
from functools   import lru_cache, reduce
from statistics  import mean, median, mode, stdev, variance
from itertools   import (permutations, combinations, groupby, cycle, chain, zip_longest, takewhile, dropwhile, count as count_from)
from heapq       import heappush, heappop, nsmallest
from operator    import iand, ior, ilshift, irshift

In [21]:
def Input(day):
    "Open this day's input file"
    return open('inputs/day{}.txt'.format(day))

cat = ''.join

## General Programming Tips

1. Use assert to test function and show usage
2. 

# [Day 1: No Time for a Taxicab](https://adventofcode.com/2016/day/1)

1. Use regex to parse is more robust than using str.split();
2. Use complex number to represent points on 2D plane is handy, turn left is equivalent to multiply 1j, turn right is equivalent to multiply -1j.

In [13]:
N, S, E, W = 1j, -1j, 1, -1 # unit vector in each direction
def parse(text):
    "Return a list of (turn, distance) pairs text of form 'R2, L32, ...'"
    turns = dict(L=N, R=S)
    return [(turns[RL], int(d))
            for (RL, d) in re.findall(r'(R|L)(\d+)', text)]
    
def taxicab_distance(point):
    "Taxicab distance between point and the origin."
    return abs(point.real) + abs(point.imag)
    
def travel_distance(moves):
    "After following moves, how far away from the origin do we end up?"
    loc, heading = 0, N
    for turn, d in parse(moves):
        heading *= turn
        loc += heading * d
    return taxicab_distance(loc)

def visited_twice(moves):
    "Following moves in text, find the first location we visited twice, and return its distance to origin"
    loc, heading = 0, N
    visited = {loc}
    for turn, d in parse(moves):
        heading *= turn
        for _ in range(d):
            loc += heading
            if loc in visited:
                return taxicab_distance(loc)
            visited.add(loc)

assert travel_distance('R2, L3') == 5
assert travel_distance('R2, R2, R2') == 2
assert visited_twice("R8, R0, R1") == 7
# with open('inputs/day1.txt', 'r') as f:
#     text = f.read()
text = Input(1).read()
print(travel_distance(text))
print(visited_twice(text))

300.0
159.0


# [Day 2: Bathroom Security](https://adventofcode.com/2016/day/2)

1. A list of string could be seen as matrix of characters;
2. Use margin instead of bounday check;
3. Consise if...else to improve readibility;
4. Use yield instead of return a list.

In [22]:
Keypad = str.split
keypad = """
.....
.123.
.456.
.789.
.....
"""

off = '.' # improve readibility

keypad = Keypad(keypad)

assert keypad[2][2] == '5'

def decode(keyboard, instructions, x=2,y=2):
    "Following instructions and decode each line to a key"
    for inst in instructions:
        for ch in inst:
            x, y = move(keyboard, ch, x, y)
        yield keypad[y][x]
    
def move(keypad, ch, x, y):
    "Update position (x, y) on keypad according to character (L/R/U/D)"
    if   ch == 'L' and keypad[y][x - 1] is not off: x -= 1
    elif ch == 'R' and keypad[y][x + 1] is not off: x += 1
    elif ch == 'U' and keypad[y - 1][x] is not off: y -= 1
    elif ch == 'D' and keypad[y + 1][x] is not off: y += 1
    return x, y

assert move(keypad, 'U', 2, 2) == (2, 1)
assert cat(decode(keypad, "ULL RRDDD LURDL UUUUD".split())) == '1985'

instructions = Input(2)
print(cat(decode(keypad, instructions)))

33444


In [23]:
keypad = """
.......
...1...
..234..
.56789.
..ABC..
...D...
.......
"""

keypad = Keypad(keypad)

print(cat(decode(keypad, Input(2))))

446A6


# [Day 3: Squares With Three Sides](https://adventofcode.com/2016/day/3)

1. Use numpy loadtxt to read file to matrix, do not forget to set dtype

In [25]:
def is_triangle(sides):
    "Do this side lengths form a valid triangle?"
    x, y, z = sorted(sides)
    return z < x + y

In [27]:
m = np.loadtxt('inputs/day3.txt', dtype=np.int64)
print(sum(map(is_triangle, m)))

917


In [29]:
m = np.loadtxt('inputs/day3.txt', dtype=np.int64).flatten('F')
m = m.reshape((len(m)//3, 3))

In [31]:
print(sum(map(is_triangle, m)))

1649


# [Day 4: Security Through Obscurity](https://adventofcode.com/2016/day/4)
1. A drawback of using nested function is hard to test

In [35]:
def parse(line):
    "Return (name, sector, checksum)."
    return re.match(r"(.+)-(\d+)\[([a-z]+)\]", line).groups()
assert parse('aaaaa-bbb-z-y-x-123[abxyz]') == ('aaaaa-bbb-z-y-x', '123', 'abxyz')

In [39]:
def sector(line):
    "Return the sector number if valid, otherwise return 0."
    name, sector, checksum = parse(line)
    return int(sector) if valid(name, checksum) else 0

def valid(name, checksum):
    "Determine if name is valid according to checksum."
    counts = Counter(name.replace('-', ''))
    letters = sorted(counts, key=lambda x : (-counts[x], x))
    return checksum == cat(letters[:5])

assert sector('aaaaa-bbb-z-y-x-123[abxyz]') == 123
print(sum(map(sector, Input(4))))

361724


# Day 5: How About a Nice Game of Chess?

In [13]:
import hashlib
s = "wtnhxymk"
i = 0
ans = ''
while len(ans) < 8:
    m = hashlib.md5()
    m.update((s + str(i)).encode('utf-8'))
    code = m.hexdigest()
    if code.startswith('00000'):
        ans += code[5]
    i += 1
print(ans)

2414bc77


In [14]:
s = "wtnhxymk"
i = 0
ans = [None] * 8
cnt = 0
while cnt < 8:
    m = hashlib.md5()
    m.update((s + str(i)).encode('utf-8'))
    code = m.hexdigest()
    if code.startswith('00000'):
        if code[5].isdigit() and int(code[5]) < 8 and ans[int(code[5])] is None:
            ans[int(code[5])] = code[6]
            cnt += 1
    i += 1
print(ans)

['4', '3', '7', 'e', '6', '0', 'f', 'c']


In [15]:
print(''.join(ans))

437e60fc


# Day 6: Signals and Noise

In [16]:
counters = [Counter() for _ in range(8)]
with open('inputs/day6.txt', 'r') as f:
    for line in f:
        for counter, ch in zip(counters, line):
            counter[ch] += 1
print(''.join([counter.most_common(1)[0][0] for counter in counters]))
print(''.join([counter.most_common()[-1][0] for counter in counters]))

mshjnduc
apfeeebz


# Day 7: Internet Protocol Version 7

In [17]:
def support_TLS(line):
    has = False
    for i in range(len(line) - 3):
        four = line[i:i + 4]
        if four[0] == four[3] and four[1] == four[2] and four[0] != four[1]:
            j = i - 1
            while j >= 0 and line[j] != ']':
                if line[j] == '[':
                    return False
                j -= 1
            has = True
    return has

In [18]:
with open('inputs/day7.txt', 'r') as f:
    print(sum(support_TLS(line) for line in f))

105


In [19]:
def support_SSL(line):
    outs = set()
    ins = set()
    for i in range(len(line) - 2):
        three = line[i:i + 3]
        if three[0] == three[2] != three[1]:
            j = i - 1
            while j >= 0 and line[j] != ']':
                if line[j] == '[':
                    ins.add(three)
                    break
                j -= 1
            else:
                outs.add(three)
    return any((three[1] + three[0] + three[1]) in outs for three in ins)
with open('inputs/day7.txt', 'r') as f:
    print(sum(support_SSL(line) for line in f))

258


# Day 8: Two-Factor Authentication

In [20]:
m, n = 6, 50
grid = [[0] * n for _ in range(m)]
with open('inputs/day8.txt', 'r') as f:
    for i, line in enumerate(f):
#         print(i)
        if line.startswith('rect'):
            _, item = line.split()
            c, r = item.split('x')
            for i in range(int(r)):
                for j in range(int(c)):
                    grid[i][j] = 1
        else:
            *_, rc, _, num = line.split()
            num = int(num)
            if rc[0] == 'y':
                # rotate row
                r = int(rc[2:])
                num = num % n
                if num != 0:
                    grid[r] = grid[r][-num:] + grid[r][:-num]
            else:
                # rotate col
                c =int(rc[2:])
                col = [grid[i][c] for i in range(m)]
                num = num % m
                if num != 0:
                    col = col[-num:] + col[:-num]
                    for i in range(m):
                        grid[i][c] = col[i]
print(sum(map(sum, grid)))

119


In [21]:
len(grid), len(grid[0])

(6, 50)

In [22]:
cols = zip(*grid)
iters = [iter(cols)] * 5
code = list(zip(*iters))
[sum(map(sum, item)) for item in code]

[12, 11, 14, 11, 11, 11, 12, 13, 12, 12]

In [23]:
for letter in code:
    for row in zip(*letter):
        print(row)
    print()

(1, 1, 1, 1, 0)
(0, 0, 0, 1, 0)
(0, 0, 1, 0, 0)
(0, 1, 0, 0, 0)
(1, 0, 0, 0, 0)
(1, 1, 1, 1, 0)

(1, 1, 1, 1, 0)
(1, 0, 0, 0, 0)
(1, 1, 1, 0, 0)
(1, 0, 0, 0, 0)
(1, 0, 0, 0, 0)
(1, 0, 0, 0, 0)

(1, 0, 0, 1, 0)
(1, 0, 0, 1, 0)
(1, 1, 1, 1, 0)
(1, 0, 0, 1, 0)
(1, 0, 0, 1, 0)
(1, 0, 0, 1, 0)

(1, 1, 1, 1, 0)
(1, 0, 0, 0, 0)
(1, 1, 1, 0, 0)
(1, 0, 0, 0, 0)
(1, 0, 0, 0, 0)
(1, 0, 0, 0, 0)

(0, 1, 1, 1, 0)
(1, 0, 0, 0, 0)
(1, 0, 0, 0, 0)
(0, 1, 1, 0, 0)
(0, 0, 0, 1, 0)
(1, 1, 1, 0, 0)

(1, 1, 1, 1, 0)
(1, 0, 0, 0, 0)
(1, 1, 1, 0, 0)
(1, 0, 0, 0, 0)
(1, 0, 0, 0, 0)
(1, 0, 0, 0, 0)

(0, 1, 1, 0, 0)
(1, 0, 0, 1, 0)
(1, 0, 0, 1, 0)
(1, 0, 0, 1, 0)
(1, 0, 0, 1, 0)
(0, 1, 1, 0, 0)

(0, 1, 1, 0, 0)
(1, 0, 0, 1, 0)
(1, 0, 0, 0, 0)
(1, 0, 1, 1, 0)
(1, 0, 0, 1, 0)
(0, 1, 1, 1, 0)

(1, 1, 1, 0, 0)
(1, 0, 0, 1, 0)
(1, 0, 0, 1, 0)
(1, 1, 1, 0, 0)
(1, 0, 0, 0, 0)
(1, 0, 0, 0, 0)

(0, 1, 1, 0, 0)
(1, 0, 0, 1, 0)
(1, 0, 0, 1, 0)
(1, 0, 0, 1, 0)
(1, 0, 0, 1, 0)
(0, 1, 1, 0, 0)



# Day 9: Explosives in Cyberspace

In [24]:
def decompress_v1(line):
    l = line.find('(')
    if l != -1:
        r = line.find(')', l)
        length, num = line[l+1:r].split('x')
        num = int(num)
        length = int(length)
        return l + length * num + decompress_v1(line[r + 1 + length:])
    else:
        return len(line)
def decompress_v2(line):
    l = line.find('(')
    if l != -1:
        r = line.find(')', l)
        length, num = line[l+1:r].split('x')
        num = int(num)
        length = int(length)
        return l + num * decompress_v2(line[r+1:r+1+length]) + decompress_v2(line[r + 1 + length:])
    else:
        return len(line)
with open('inputs/day9.txt', 'r') as f:
    line = f.read()
print(decompress_v1(line))
print(decompress_v2(line))

183269
11317278863


# Day 10: Balance Bots

In [25]:
bots = defaultdict(list)
operators = defaultdict(list)
bins = defaultdict(list)
with open('inputs/day10.txt', 'r') as f:
    lines = f.readlines()
    inits = [line.strip() for line in lines if line.startswith('value')]
    ops = [line.strip() for line in lines if line.startswith('bot')]
for init in inits:
    _, num, *_, bot_num = init.split()
    bots[int(bot_num)].append(int(num))
for key in bots:
    bots[key].sort()
for op in ops:
    items = op.split()
    assert(int(items[1]) not in operators)
    operators[int(items[1])] = [items[5:7], items[-2:]]

In [26]:
def bot_op(dest, dest_num, num):
    if dest == 'output':
        bins[dest_num].append(num)
    else:
        bots[dest_num].append(num)
        bots[dest_num].sort()
flag = True
while flag:
    flag = False
    for bot_num in bots:
        if len(bots[bot_num]) == 2 and bot_num in operators:
            assert(bot_num in operators)
            low, high = operators.pop(bot_num)
            low, low_dest = low
            high, high_dest = high
            low_num, high_num = bots[bot_num]
            if low_num == 17 and high_num == 62:
                print(bot_num)
            bot_op(low, int(low_dest), low_num)
            bot_op(high, int(high_dest), high_num)
            flag = True
            break

In [27]:
for key in bots:
    if bots[key] == [17, 61]:
        print(key)

86


In [28]:
bins[0][0] * bins[1][0] * bins[2][0]

22847

# Day 11: Radioisotope Thermoelectric Generators

# Day 12: Leonardo's Monorail

In [43]:
with open('inputs/day12.txt', 'r') as f:
    ops = [line.strip().split() for line in f]
def get_value(s, registers):
    if s.isalpha():
        return registers[s]
    else:
        return int(s)
def run(registers, ops):
    i = 0
    while i < len(ops):
        op, *values = ops[i]
        if op == 'inc':
            registers[values[0]] += 1
        elif op == 'dec':
            registers[values[0]] -= 1
        elif op == 'cpy':
            y = values[1]
            v = get_value(values[0], registers)
            registers[y] = v
        else:
            v = get_value(values[1], registers)
            a = get_value(values[0], registers)
            if a != 0:
                i += v
                continue
        i += 1
    return registers
registers = {'a' : 0, 'b' : 0, 'c' : 0, 'd' : 0}
registers = run(registers, ops)
print(registers['a'])
registers = {'a' : 0, 'b' : 0, 'c' : 1, 'd' : 0}
registers = run(registers, ops)
print(registers['a'])

318003
9227657


# Day 13: A Maze of Twisty Little Cubicles

In [29]:
def maze_func(x, y):
    return x * x + 3 * x + 2 * x * y + y + y * y
def is_open(d, x, y, num):
    if (x, y) in d:
        return d[x, y]
    else:
        m = maze_func(x, y) + num
        ones = bin(m).count('1')
        d[x, y] = (ones % 2 == 0)
        return d[x, y]
    
def maze_bfs(start, target, num):
    dq = deque([start])
    visited = {start}
    step = 0
    d = {}
    while dq:
        size = len(dq)
        for _ in range(size):
            x, y = dq.popleft()
#             print(x, y)
            if (x, y) == target:
                return step
            for dx, dy in (-1, 0), (1, 0), (0, -1), (0, 1):
                newx, newy = x + dx, y + dy
                if 0 <= newx and 0 <= newy and is_open(d, newx, newy, num) and (newx, newy) not in visited:
                    visited.add((newx, newy))
                    dq.append((newx, newy))
        step += 1
num = 1362 # 10
start = (1, 1)
target = (31, 39) # (7, 4)
print(maze_bfs(start, target, num))

82


In [30]:
def maze_bfs_part2(start, nsteps, num):
    dq = deque([start])
    visited = {start}
    step = 0
    d = {}
    cnt = 0
    while dq and step <= nsteps:
        size = len(dq)
        for _ in range(size):
            x, y = dq.popleft()
            cnt += 1
            for dx, dy in (-1, 0), (1, 0), (0, -1), (0, 1):
                newx, newy = x + dx, y + dy
                if 0 <= newx and 0 <= newy and is_open(d, newx, newy, num) and (newx, newy) not in visited:
                    visited.add((newx, newy))
                    dq.append((newx, newy))
        step += 1
    return cnt
print(maze_bfs_part2((1, 1), 50, num))

138


# Day 14: One-Time Pad

In [31]:
import hashlib

@lru_cache(None)
def md5_code(s, i, times=1):
    string = s + str(i)
    for _ in range(times):
        m = hashlib.md5()
        m.update(string.encode('utf-8'))
        string = m.hexdigest()
    return string

def gen_key(s, times):
    i = 0
    while True:
        code = md5_code(s, i, times)
        for j in range(len(code) - 2):
            if code[j] == code[j + 1] == code[j + 2]:
                five = code[j] * 5
                k = i + 1
                while k < i + 1001:
                    if five in md5_code(s, k, times):
                        yield i
                        break
                    k += 1
                break
        i += 1

In [32]:
s = "ihaygndm"
genor = gen_key(s, times=1)
print(list(itertools.islice(genor, 64))[-1])

15035


In [33]:
s = "ihaygndm"
genor = gen_key(s, times=2017)
print(list(itertools.islice(genor, 64))[-1])

19968


# [Day 15: Timing is Everything](https://adventofcode.com/2016/day/15)

In [66]:
Disc = namedtuple('Disc', 'length, start_time, start_pos')
discs = []
with open('inputs/day15.txt', 'r') as f:
    for line in f:
        discs.append(Disc(*map(int, re.findall(r'[0-9]+', line)[1:])))

In [67]:
print(discs)

[Disc(length=7, start_time=0, start_pos=0), Disc(length=13, start_time=0, start_pos=0), Disc(length=3, start_time=0, start_pos=2), Disc(length=5, start_time=0, start_pos=2), Disc(length=17, start_time=0, start_pos=0), Disc(length=19, start_time=0, start_pos=7)]


In [68]:
def push_time(discs):
    # init discs
    iters = []
    for idx, disc in enumerate(discs, 1):
        lst = list(range(disc.length))
        assert(disc.start_time == 0)
        iterable = cycle(lst)
        for _ in range(disc.start_pos + idx):
            next(iterable)
        iters.append(iterable)
    for i in count_from(0):
        vs = [next(iterable) for iterable in iters]
        if not any(vs):
            return i
print(push_time(discs))

121834


In [70]:
discs.append(Disc(11, 0, 0))
print(push_time(discs))

3208099


# [Day 16: Dragon Checksum](https://adventofcode.com/2016/day/16)

In [46]:
# 11100010111110100
def process(a):
    lst = list(a)
    b = ''.join('0' if ch == '1' else '1' for ch in reversed(lst))
    return a + '0' + b
assert(process('1') == '100')
assert(process('111100001010') == '1111000010100101011110000')
def checksum(a):
    iters = [iter(a)] * 2
    lst = []
    for a, b in zip(*iters):
        if a == b:
            lst.append('1')
        else:
            lst.append('0')
    return ''.join(lst) if len(lst) % 2 == 1 else checksum(lst)
assert(checksum('110010110100') == '100')

In [47]:
def dragon_checksum(length, init):
    while len(init) < length:
        init = process(init)
    init = init[:length]
    return checksum(init)
assert(dragon_checksum(20, '10000') == '01100')

In [48]:
print(dragon_checksum(272, '11100010111110100'))

10100011010101011


In [49]:
print(dragon_checksum(35651584, '11100010111110100'))

01010001101011001


# Day 17: Two Steps Forward

In [34]:
dirs = {(-1, 0) : 'L', (1, 0) : 'R', (0, 1) : 'D', (0, -1) : 'U'}
def shortest_path(passcode):
    def door_conditions(path):
        m = hashlib.md5()
        m.update((passcode + path).encode('utf-8'))
        ans = m.hexdigest()[:4]
        return [item in 'bcdef' for item in ans] # up down left right
    dq = deque([(1, 1, '')])
    seen = {''}
    while dq:
        x, y, path = dq.popleft()
        if x == y == 4:
            return path
        for (dx, dy), valid in zip(((0, -1), (0, 1), (-1, 0), (1, 0)), door_conditions(path)):
            if valid and 0 < x + dx <= 4 and 0 < y + dy <= 4 and path + dirs[(dx, dy)] not in seen:
                seen.add(path + dirs[(dx, dy)])
                dq.append((x + dx, y + dy, path + dirs[(dx, dy)]))

In [35]:
assert(shortest_path('ihgpwlah') == 'DDRRRD')
assert(shortest_path('kglvqrro') == 'DDUDRLRRUDRD')
assert(shortest_path('ulqzkmiv') == 'DRURDRUDDLLDLUURRDULRLDUUDDDRR')

In [36]:
passcode = 'qzthpkfp'
print(shortest_path(passcode))

RDDRLDRURD


In [37]:
def longest_path(passcode):
    def door_conditions(path):
        m = hashlib.md5()
        m.update((passcode + path).encode('utf-8'))
        ans = m.hexdigest()[:4]
        return [item in 'bcdef' for item in ans] # up down left right
    dq = deque([(1, 1, '')])
    seen = {''}
    mx_length = 0
    while dq:
        x, y, path = dq.popleft()
        if x == y == 4:
            mx_length = max(mx_length, len(path))
            continue
        for (dx, dy), valid in zip(((0, -1), (0, 1), (-1, 0), (1, 0)), door_conditions(path)):
            if valid and 0 < x + dx <= 4 and 0 < y + dy <= 4 and path + dirs[(dx, dy)] not in seen:
                seen.add(path + dirs[(dx, dy)])
                dq.append((x + dx, y + dy, path + dirs[(dx, dy)]))
    return mx_length

In [38]:
assert(longest_path('ihgpwlah') == 370)
assert(longest_path('kglvqrro') == 492)
assert(longest_path('ulqzkmiv') == 830)

In [39]:
print(longest_path('qzthpkfp'))

448


# Day 18: Like a Rogue

In [46]:
rules = {'^^.', '.^^', '^..', '..^'}
def next_row(row, rules):
    n = len(row)
    ans = ''
    for i, center in enumerate(row):
        left = '.' if i == 0 else row[i - 1]
        right = '.' if i == n - 1 else row[i + 1]
        if (left + center + right) in rules:
            ans += '^'
        else:
            ans += '.'
    return ans
def day18(row, rules, nrows = 40):
    cnt = 0
    for i in range(nrows):
        cnt += sum(ch == '.' for ch in row)
        row = next_row(row, rules)
    return cnt

In [50]:
assert(day18('.^^.^.^^^^', rules, 10) == 38)
with open('inputs/day18.txt', 'r') as f:
    print(day18(f.read().strip(), rules, 40))
#     print(day18(f.read().strip(), rules, 400000)) # 20006289

2013


# Day 19: An Elephant Named Joseph

In [57]:
def day19(n):
    lst = [1] * n
    cur = 0
    while True:
        if lst[cur] > 0:
            nxt = (cur + 1) % n
            while lst[nxt] == 0:
                nxt = (nxt + 1) % n
            lst[cur] += lst[nxt]
            lst[nxt] = 0
            if lst[cur] == n:
                return cur + 1
            cur = (nxt + 1) % n
        else:
            cur = (cur + 1) % n

In [59]:
assert(day19(5) == 3)
print(day19(3017957))

1841611


# Day 20: Firewall Rules

In [67]:
with open('inputs/day20.txt', 'r') as f:
    intervals = []
    for line in f:
        left, right = line.strip().split('-')
        intervals.append((int(left), int(right)))
def day20(low, high, intervals):
    # compute the lowest allowed IP and number of allowed IPs
    lowest = None
    cnt = 0
    for start, end in sorted(intervals):
        if start > low:
            if lowest is None:
                lowest = low
            cnt += start - low
        low = max(end + 1, low)
    if lowest is None:
        lowest = low
    return lowest, cnt + (high - low + 1)
print(day20(0, 4294967295, intervals))

(19449262, 119)


# [Day 21: Scrambled Letters and Hash](https://adventofcode.com/2016/day/21)

In [24]:
with open('inputs/day21.txt', 'r') as f:
    ops = [line.strip() for line in f]

In [40]:
def scramble_password(string, ops, reverse=False, verbose=False):
    ans = list(string)
    n = len(ans)
    def move_right(ans, step):
        step = step % len(ans)
        return ans[-step:] + ans[:-step]
    extract_nums = lambda line : map(int, re.findall(r'-?[0-9]+', line))
    for op in ops:
        if op.startswith('rotate left') or op.startswith('rotate right'):
            step, = re.findall(r'-?[0-9]+', op)
            if 'right' in op:
                step = int(step) % n
            else:
                step = n - int(step) % n
            if reverse: step = n - step
            ans = move_right(ans, step)
        elif op.startswith('swap position'):
            x, y = extract_nums(op)
            ans[x], ans[y] = ans[y], ans[x]
        elif op.startswith('swap letter'):
            _, _, x, *_, y = op.split()
            ix, iy = ans.index(x), ans.index(y)
            ans[ix], ans[iy] = ans[iy], ans[ix]
        elif op.startswith('reverse'):
            x, y = extract_nums(op)
            ans = ans[:x] + ans[x:y+1][::-1] + ans[y+1:]
        elif op.startswith('move'):
            x, y = extract_nums(op)
            if reverse:
                x, y = y, x
            item = ans.pop(x)
            ans.insert(y, item)
        else:
            *_, x = op.rsplit(' ', 1)
            if not reverse:
                idx = ans.index(x)
                step = idx + 2 if idx >= 4 else idx + 1
                ans = move_right(ans, step)
            else:
                idx = ans.index(x)
                i = 0 # previous idx
                while i < n:
                    step = i + 2 if i >= 4 else i + 1
                    if (i + step) % n == idx:
                        break
                    i += 1
                step = (idx - i) if idx >= i else n - (i - idx)
                ans = move_right(ans, n - step)
        if verbose: print(ans)
    return ''.join(ans)

In [41]:
string = "abcdefgh"
print(scramble_password(string, ops, verbose=False))

gbhafcde


In [42]:
string = "fbgdceah"
print(scramble_password(string, ops[::-1], reverse=True, verbose=False))

bcfaegdh


# Day 22: Grid Computing

In [45]:
# root@ebhq-gridcenter# df -h
# Filesystem              Size  Used  Avail  Use%
nodes = []
with open('inputs/day22.txt', 'r') as f:
    lines = [line.strip() for line in f if line.startswith('/dev')]
print(lines[0])
for line in lines:
    path, size, used, avail, _ = line.split()
    _, x, y = path.split('-')
    nodes.append(tuple(map(int, [x[1:], y[1:], used[:-1], avail[:-1]])))
print(nodes[0])

/dev/grid/node-x0-y0     91T   66T    25T   72%
(0, 0, 66, 25)


In [46]:
# number of pairs of node used fit into avail
print(sum(used > 0 and used <= avail for 
          (_, _, used, _), (_, _, _, avail) in permutations(nodes, 2)))

901


In [47]:
# this is a A* search
xmax = max(x for x, _, _, _ in nodes)
ymax = max(y for _, y, _, _ in nodes)
m = ymax + 1
n = xmax + 1
from copy import deepcopy
# BFS
grid = [[None] * n for _ in range(m)]
for x, y, used, avail in nodes:
    grid[y][x] = [used, avail + used]
empty = next((node for node in nodes if node[2] == 0))

In [48]:
print(empty)

(35, 18, 0, 85)


In [49]:
start = ((xmax, 0), (empty[0], empty[1]))
def distance(state):
    data_x, data_y = state[0]
    return abs(data_x) + abs(data_y)
def Path(previous, s):
    return ([] if (s is None) else Path(previous, previous[s]) + [s])
pq = [(distance(start), start)]
previous = {start : None}
path_cost = {start : 0}
while pq:
    (f, s) = heappop(pq)
    if distance(s) == 0:
        path = Path(previous, s)
        break
    data_x, data_y = s[0]
    empty_x, empty_y = s[1]
    for dx, dy in (-1, 0), (1, 0), (0, -1), (0, 1):
        newx, newy = empty_x + dx, + empty_y + dy
        if 0 <= newy < len(grid) and 0 <= newx < len(grid[0]):
            used, size = grid[newy][newx][0], grid[empty_y][empty_x][1]
            if used <= size:
                if newx == data_x and newy == data_y:
                    new_data_x, new_data_y = empty_x, empty_y
                else:
                    new_data_x, new_data_y = data_x, data_y
                news = ((new_data_x, new_data_y), (newx, newy))
                newcost = path_cost[s] + 1
                if news not in path_cost or newcost < path_cost[news]:
                    heappush(pq, (newcost + distance(news), news))
                    path_cost[news] = newcost
                    previous[news] = s
print(len(path) - 1)

238


In [36]:
# # the key is how to encode the state, and from a state generate next state
# xmax = max(x for x, _, _, _ in nodes)
# ymax = max(y for _, y, _, _ in nodes)
# m = ymax + 1
# n = xmax + 1
# from copy import deepcopy
# # BFS
# grid = [[None] * n for _ in range(m)]
# for x, y, used, avail in nodes:
#     grid[y][x] = [used, avail + used]
# def encode_grid(grid, x, y):
#     lst = []
#     for row in grid:
#         for a, b in row:
#             lst.append(a)
#             lst.append(b)
#     lst.append(x)
#     lst.append(y)
#     return tuple(lst)
# x, y = xmax, 0
# dq = deque([(grid, (x, y), 0)])
# visited = {encode_grid(grid, xmax, 0)}
# while dq:
#     grid, (x, y), dis = dq.popleft()
#     if x == y == 0:
#         print(dis)
#         break
#     for i, row in enumerate(grid):
#         for j, (used, total) in enumerate(row):
#             for di, dj in (-1, 0), (1, 0), (0, -1), (0, 1):
#                 newi, newj = i + di, j + dj
#                 if 0 <= newi < len(grid) and 0 <= newj < len(grid[0]):
#                     if grid[newi][newj][1] - grid[newi][newj][0] >= used:
#                         # it can move
#                         if i == y and j == x:
#                             newx, newy = newj, newi
#                         else:
#                             newx, newy = x, y
#                         new_grid = deepcopy(grid)
#                         new_grid[newi][newj][0] += new_grid[i][j][0]
#                         new_grid[i][j][0] = 0
#                         code = encode_grid(new_grid, newx, newy)
#                         if code not in visited:
#                             visited.add(code)
#                         dq.append((new_grid, (newx, newy), dis + 1))

7


# [Day 23: Safe Cracking](https://adventofcode.com/2016/day/23) 

In [98]:
with open('inputs/day23.txt', 'r') as f:
    program = [[(x if x.isalpha() else int(x))
                for x in line.strip().split()]
               for line in f]
print(program[0], program[-1])
print(program)

['cpy', 'a', 'b'] ['jnz', 'c', -5]
[['cpy', 'a', 'b'], ['dec', 'b'], ['cpy', 'a', 'd'], ['cpy', 0, 'a'], ['cpy', 'b', 'c'], ['inc', 'a'], ['dec', 'c'], ['jnz', 'c', -2], ['dec', 'd'], ['jnz', 'd', -5], ['dec', 'b'], ['cpy', 'b', 'c'], ['cpy', 'c', 'd'], ['dec', 'd'], ['inc', 'c'], ['jnz', 'd', -2], ['tgl', 'c'], ['cpy', -16, 'c'], ['jnz', 1, 'c'], ['cpy', 78, 'c'], ['jnz', 70, 'd'], ['inc', 'a'], ['inc', 'd'], ['jnz', 'd', -2], ['inc', 'c'], ['jnz', 'c', -5]]


In [99]:
def run(registers, program):
    pc = 0
    n = len(program)
    fetch = lambda x : x if type(x) == int else registers[x]
    while 0 <= pc < n:
        inst = program[pc]
        ins, x, y = inst[0], inst[1], inst[-1]
        if ins == 'inc':
            registers[x] += 1
        elif ins == 'dec':
            registers[x] -= 1
        elif ins == 'jnz':
            val = fetch(x)
            if val != 0:
                pc += fetch(y)
                continue
        elif ins == 'tgl':
            toggle(program, pc + fetch(x))
        else:
            val = fetch(x)
            registers[y] = val
        pc += 1
    return registers['a']
def toggle(program, i):
    if 0 <= i < len(program):
        inst = program[i]
        inst[0] = ('dec' if inst[0] == 'inc' else
                   'inc' if len(inst) == 2 else
                   'cpy' if inst[0] == 'jnz' else
                   'jnz')

In [100]:
registers = dict(a=7, b=0, c=0, d=0)
run(registers, program)

10500

# Day 24: Air Duct Spelunking

In [24]:
with open('inputs/day24.txt', 'r') as f:
    grid = [list(line.strip()) for line in f]

In [25]:
locs = {}
for i, row in enumerate(grid):
    for j, ch in enumerate(row):
        if ch.isdigit():
            locs[ch] = (i, j)
distances = {(i, j) : float('inf') for i, j in permutations(locs, 2)}
def bfs(grid, start, locs, distances):
    "Do BFS from start and update its distance to all other nums"
    x, y = locs[start]
    dq = deque([(x, y, 0)])
    visited = {(x, y)}
    while dq:
        i, j, dis = dq.popleft()
        for di, dj in (-1, 0), (1, 0), (0, -1), (0, 1):
            newi, newj = i + di, j + dj
            if grid[newi][newj] != '#' and (newi, newj) not in visited:
                if grid[newi][newj].isdigit() and distances[start, grid[newi][newj]] > dis + 1:
                    distances[start, grid[newi][newj]] = distances[grid[newi][newj], start] = dis + 1
                visited.add((newi, newj))
                dq.append((newi, newj, dis + 1))
for i, row in enumerate(grid):
    for j, ch in enumerate(row):
        if ch.isdigit():
            bfs(grid, ch, locs, distances)

In [26]:
others = list(locs.keys() - {'0'})

In [29]:
def travel_distance(path, distances, closed=False):
    ans = sum(distances[a, b] for a, b in zip(path, path[1:]))
    return ans + distances[path[-1], path[0]] if closed else ans
print(min(travel_distance(['0'] + list(path), distances) for path in permutations(others)))
print(min(travel_distance(['0'] + list(path), distances, closed=True) for path in permutations(others)))

462
676


# [Day 25: Clock Signal](https://adventofcode.com/2016/day/25)

In [73]:
program = []
with open('inputs/day25.txt', 'r') as f:
    for line in f:
        op, *opands = re.findall(r'[-\w]+', line)
        program.append((op, [int(opand) if re.match(r'-?[0-9]+', opand) else opand 
                             for opand in opands] ))

In [74]:
print(program)

[('cpy', ['a', 'd']), ('cpy', [4, 'c']), ('cpy', [633, 'b']), ('inc', ['d']), ('dec', ['b']), ('jnz', ['b', -2]), ('dec', ['c']), ('jnz', ['c', -5]), ('cpy', ['d', 'a']), ('jnz', [0, 0]), ('cpy', ['a', 'b']), ('cpy', [0, 'a']), ('cpy', [2, 'c']), ('jnz', ['b', 2]), ('jnz', [1, 6]), ('dec', ['b']), ('dec', ['c']), ('jnz', ['c', -4]), ('inc', ['a']), ('jnz', [1, -7]), ('cpy', [2, 'b']), ('jnz', ['c', 2]), ('jnz', [1, 4]), ('dec', ['b']), ('dec', ['c']), ('jnz', [1, -4]), ('jnz', [0, 0]), ('out', ['b']), ('jnz', ['a', -19]), ('jnz', [1, -21])]


In [75]:
def run(a, program):
    "yield value one by one"
    registers = defaultdict(int)
    registers['a'] = a
    pc = 0
    n = len(program)
    fetch = lambda x : x if type(x) == int else registers[x]
    while 0 <= pc < n:
        ins, opands = program[pc]
        if ins == 'inc':
            registers[opands[0]] += 1
        elif ins == 'dec':
            registers[opands[0]] -= 1
        elif ins == 'out':
            yield registers[opands[0]]
        elif ins == 'jnz':
            val = fetch(opands[0])
            if val != 0:
                pc += opands[1]
                continue
        else:
            assert(ins == 'cpy')
            val = fetch(opands[0])
            registers[opands[1]] = val
        pc += 1

In [76]:
print(list(itertools.islice(run(100, program), 10)))
print(list(itertools.islice(cycle([0, 1]), 10)))

[0, 0, 0, 1, 0, 0, 1, 0, 0, 1]
[0, 1, 0, 1, 0, 1, 0, 1, 0, 1]


In [77]:
def probe_a(program, times = 1000):
    "compute the lowest positive integer of a"
    a = 1
    while True:
        g = run(a, program)
        for v1, v2 in itertools.islice(zip(g, cycle((0, 1))), times):
            if v1 != v2:
                break
        else:
            break
        a += 1
    return a

In [78]:
probe_a(program, times=1000)

198